# RSI
韦尔斯-威尔德 在1978年6月发表一种衡量证券自身内在相对强度的指标，Relative Strength Index，简称RSI，中文名为“相对强弱指标”。
RSI指标用来判断股票的买入和卖出情况，从而预测未来股票价格走势。

$$
\begin{aligned}
& RSI = 100 * \frac{\frac{UP}{DOWN}}{1 + {\frac{UP}{DOWN}}} \\
&  = 100* \frac{UP}{UP+DOWN}\\

\end{aligned}
$$

RSI 表示相对强弱的指标值，若t表示时期，UP表示t期内股价上涨幅度的平均值，DOWN表示t期内股价下跌幅度的平均值。

# RSI 指标实例

使用Python编写RSI，并运用RSI值制定交易策略。策略编写思路：
1. 数据准备：获取股票数据，提取出收盘价数据；

2. 捕捉买卖点：计算RSI6 和 RSI24的值

		当RSI6 > 80时，卖出信号；当RSI6 向下穿过 RSI24 时，为卖出信号；
		
		当RSI6 < 20时，买入信号；当RSI6 向上穿过 RSI24 时，为买入信号。
		
3. 执行交易策略：按照RSI买卖点买入或卖出股票；
4. 回测：对交易策略进行投资评价。

In [1]:
!jupyter kernelspec list

Available kernels:
  python3    /Users/paul/anaconda3/envs/pybrokerlab/share/jupyter/kernels/python3
  py37lab    /usr/local/share/jupyter/kernels/py37lab


In [54]:
#vscode中使用python引入自定义模块
import sys
sys.path.append(r'/Users/paul/DSWorkspace/001量化策略/002-Project项目/lleasy')

# 1. 数据准备
from lleasy.database import SqliteDatabase as sqlitedb
from lleasy.config import Config
from lleasy.object import TradeData, BarData
import numpy as np
import pandas as pd

#获取sqlite链接
config = Config()
config_dict = config.get_config('datasource_file')
local_db_type = config_dict['local_db_type']
local_db_path = config_dict['local_db_path']

db = sqlitedb(local_db_path)


## 数据准备

In [55]:
# 获取数据
# 标的：510050
# 时间：2022-01-01 至 2023-01-01
# 频率：日行情
session = sqlitedb(local_db_path).getDBSession()
bars = session.query(BarData)\
        .where(BarData.symbol=='510050')\
        .where(BarData.datetime >= '2022-01-01')\
        .where(BarData.datetime <= '2023-01-01')\
        .all()
results = [bar.to_dict() for bar in bars]
results=pd.DataFrame(
    results,
    columns=['datetime','symbol','open','close','high','low']
)

results=results.set_index(['datetime'])
results

2023-07-23 12:55:40,893 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-07-23 12:55:40,896 INFO sqlalchemy.engine.Engine SELECT llbardata.id AS llbardata_id, llbardata.symbol AS llbardata_symbol, llbardata.interval AS llbardata_interval, llbardata.open AS llbardata_open, llbardata.close AS llbardata_close, llbardata.high AS llbardata_high, llbardata.low AS llbardata_low, llbardata.vol AS llbardata_vol, llbardata.turnover AS llbardata_turnover, llbardata.asset AS llbardata_asset, llbardata.adjust AS llbardata_adjust, llbardata.datetime AS llbardata_datetime 
FROM llbardata 
WHERE llbardata.symbol = ? AND llbardata.datetime >= ? AND llbardata.datetime <= ?
2023-07-23 12:55:40,896 INFO sqlalchemy.engine.Engine [generated in 0.00086s] ('510050', '2022-01-01', '2023-01-01')


,symbol,open,close,high,low
datetime,,,,,
2022-01-04,510050,3.270,3.247,3.270,3.205
2022-01-05,510050,3.242,3.241,3.277,3.226
2022-01-06,510050,3.229,3.193,3.238,3.180
2022-01-07,510050,3.199,3.207,3.223,3.199
2022-01-10,510050,3.208,3.216,3.223,3.185
...,...,...,...,...,...
2022-12-26,510050,2.623,2.612,2.633,2.608
2022-12-27,510050,2.639,2.642,2.650,2.626
2022-12-28,510050,2.638,2.649,2.656,2.629


## 编制策略
1. 计算RSI6 和 RSI24的值
2. 捕获信号。
   a. 获取超卖超买信号signal_bs、黄金或死叉信号signal_gd
   b. 将signal_bs 和 signal_gd 结合型成 signal信号。


In [66]:
# 捕捉买卖点：计算RSI6 和 RSI24的值
## 定义rsi函数，通过行情close价格计算rsi值
def rsi(price,period=6):
    clpriceChange = price - price.shift(1)
    clpriceChange = clpriceChange.dropna()

    indexPrice = clpriceChange.index
    upPrice = pd.Series(0,index=indexPrice)
    upPrice[clpriceChange>0] = clpriceChange[clpriceChange>0]

    downPrice = pd.Series(0,index=indexPrice)
    downPrice[clpriceChange<0] = - clpriceChange[clpriceChange<0]

    rsidata = pd.concat([price,clpriceChange,upPrice,downPrice],axis=1)
    rsidata.columns=['close','clpriceChange','upPrice','downPrice']
    rsidata = rsidata.dropna()
    
    UP = []
    DOWN = []

    for i in range(period,len(upPrice)+1):
        UP.append(np.mean(upPrice.values[(i-period):i],dtype=np.float32))
        DOWN.append(np.mean(downPrice.values[(i-period):i],dtype=np.float32))
        rsi = [100 * UP[i] / (UP[i] + DOWN[i]) for i in range(0,len(UP))]

    indexRsi = indexPrice[(period-1):]
    rsi = pd.Series(rsi,index=indexRsi)

    return rsi

# 捕获买卖信号
rsi6 = rsi(results['close'],6)
rsi24 = rsi(results['close'],24)

signal_ls_bs = []
signal_ls_gd = []

## rsi6 > 80，近期高于超买线，价格将由回落趋势，释放出卖出信号
## rsi6 < 20，近期低于超卖线，价格将由回升趋势，释放出买入信号
for i in rsi6:
    if i > 80:
        signal_ls_bs.append(-1)
    elif i < 20:
        signal_ls_bs.append(1)
    else:
        signal_ls_bs.append(0)

#获取超卖超买信号signal_bs
## signal_bs = -1。rsi6 > 80，近期高于超买线，价格将由回落趋势，释放出卖出信号
## signal_bs = 1。 rsi6 < 20，近期低于超卖线，价格将由回升趋势，释放出买入信号
signal_bs = pd.Series(signal_ls_bs,index=rsi6.index)

#黄金或死叉信号signal_gd
## 黄金交叉，RSI6 由下方向上穿过 RSI24。
## 死亡交叉，RSI6 由上方向下穿过 RSI24。

signal_gd = pd.Series(0,index=rsi24.index)
lastRSI6 = rsi6.shift(1)
lastRSI24 = rsi24.shift(1)

## signal_gd
### signal_gd = -1, 表示rsi6 从上向下穿过 rsi24，释放出卖出信号
### signal_gd = 1, 表示rsi6，从下向上穿过 rsi24，释放出买入信号
for i in rsi24.index:
    if (rsi6[i] > rsi24[i]) and (lastRSI6[i] < lastRSI24[i]):
        signal_gd[i] = 1
    elif (rsi6[i] < rsi24[i]) and (lastRSI6[i] > lastRSI24[i]):
        signal_gd[i] = -1

# 将 超卖超买 和 黄金死亡 信号结合形成最终 买卖型号 signal
## signal = 1 买入信号
## signal = -1 卖出信号
signal = signal_bs + signal_gd
signal[signal>=1] = 1
signal[signal<=-1] = -1
signal=signal.dropna()
signal


datetime
2022-02-14    0.0
2022-02-15    0.0
2022-02-16    0.0
2022-02-17    0.0
2022-02-18    0.0
             ... 
2022-12-26    1.0
2022-12-27    0.0
2022-12-28    1.0
2022-12-29    0.0
2022-12-30    0.0
Length: 218, dtype: float64

## 执行策略


In [120]:
# signal 信号出现后2天进行交易
trade = signal.shift(2)

## 回测评估
1. 策略绩效指标：收益率、年化收益率、胜率
2. 分析回测结果
3. 优化策略参数
4. 对比

In [121]:
# 计算收益率
ret = results['close'] / results['close'].shift(1) - 1

### 数据由object转换为float，否则float * object 会报错。
ret = ret[trade.index].astype(float)

## 计算买入交易收益率
buy = trade[trade == 1]
buyRet = ret[trade == 1] * buy

## 计算卖出交易收益率
sell = trade[trade == -1]
sellRet = ret[trade == -1] * sell

## 计算卖买交易合并的收益率
tradeRet = ret * trade

# 计算胜率
winRate = len(tradeRet[tradeRet>0]) / len(tradeRet[tradeRet!= 0])
winRate

0.4631578947368421

-【补充】趋势图

# 总结
1. RSI指标用于判断市场超买和超卖情况；
2. RSI线的形态分析，RSI曲线在不同区域展示出不同的买卖信号
3. 长短线RSI的黄金交叉和死亡交叉
4. 需要注意，在RSI线的走势与价格走势会出现背离。